# Final Project

CS 109 <br>
Madhu, Max, and Yunhan

## Data Collection | Yunhan

We collected all of the unstructured text of Supreme Court opinions using the CourtListener API. We decided to use the Bulk Data API instead of the REST API since the latter would have required upwards of 60,000 requests. 

In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
pd.set_option("display.width", 500)
pd.set_option("display.max_columns", 100)
pd.set_option("display.notebook_repr_html", True)
import seaborn as sns
sns.set_style("darkgrid")
sns.set_context("poster")

In [2]:
%%time

# ------ use this URL to download all opinions as multiple json files ------
# https://www.courtlistener.com/api/bulk-data/document/scotus.tar.gz

# ------ turn json files into pandas dataframe ------
import os
import glob
import json

# lists for pandas dataframe cols
raw_dict = {}
list_name,list_blocked,list_id,list_date,list_url,list_text = [],[],[],[],[],[]

# iterate through each json file, append data to col lists
for filename in glob.glob("*.json"):
    with open(filename) as json_file:
        json_data = json.load(json_file)
        list_name.append(json_data["citation"]["case_name"])
        list_blocked.append(json_data["blocked"])
        list_id.append(json_data["id"])
        list_date.append(json_data["date_filed"])
        if json_data["download_url"] != "":
            list_url.append(json_data["download_url"])
        else:
            list_url.append(json_data["citation"]["resource_uri"])
        if json_data["plain_text"] != "":
            list_text.append(json_data["plain_text"])
        else:
            list_text.append(json_data["html_with_citations"])

raw_dict["name"] = list_name
raw_dict["blocked"] = list_blocked
raw_dict["id"] = list_id
raw_dict["date"] = list_date
raw_dict["url"] = list_url
raw_dict["text"] = list_text

CPU times: user 42.1 s, sys: 16.2 s, total: 58.3 s
Wall time: 1min 26s


In [3]:
%%time

# strip html tags from text
from bs4 import BeautifulSoup

list_cleantext = []
for rawtext in list_text:
    list_cleantext.append(BeautifulSoup(rawtext).text.replace("\n",""))

raw_dict["text"] = list_cleantext

CPU times: user 5min 43s, sys: 11.5 s, total: 5min 55s
Wall time: 6min 22s


/Users/yunhanxu/anaconda/lib/python2.7/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [4]:
raw_df = pd.DataFrame(raw_dict)

In [5]:
raw_df.head()

,blocked,date,id,name,text,url
0,False,1922-05-29,100000,Morrisdale Coal Co. v. United States,259 U.S. 188 (1922)MORRISDALE COAL COMPANYv.UN...,None
1,False,1922-05-29,100001,Pine Hill Coal Co. v. United States,"259 U.S. 191 (1922)PINE HILL COAL COMPANY, INC...",None
2,False,1922-05-29,100002,Santa Fe Pacific R. Co. v. Fall,259 U.S. 197 (1922)SANTA FE PACIFIC RAILROAD C...,None
3,False,1922-05-29,100003,"Federal Baseball Club of Baltimore, Inc. v. Na...",259 U.S. 200 (1922)FEDERAL BASEBALL CLUB OF BA...,None
4,False,1922-05-29,100004,Mutual Life Ins. Co. of NY v. Liebing,259 U.S. 209 (1922)MUTUAL LIFE INSURANCE COMPA...,None


In [7]:
raw_df.to_csv("raw_clistener_data.csv", sep='\t', encoding="utf-8")